# 2. 파이썬으로 데이터 주무르기, pandas
**pandas를 활용해서 데이터프레임을 다뤄봅시다.**

1. Pandas 시작하기
    - prerequisite : Table
    - pandas import하기
   
2. Pandas로 1차원 데이터 다루기 - Series 
    - Series 선언하기
    - Series vs ndarray
    - Series vs dict
    - Series에 이름 붙이기
3. Pandas로 2차원 데이터 다루기 - dataframe
    - dataframe 선언하기
    - from csv to dataframe
    - dataframe 자료 접근하기

[수업에 사용된 covid 데이터](https://www.kaggle.com/imdevskp/corona-virus-report)

## I. pandas 시작하기

### Prerequisite : Table

- 행과 열을 이용해서 데이터를 저장하고 관리하는 자료구조 (컨테이너)
- 주로 행은 개체, 열은 속성을 나타냄

### Pandas 시작하기

`import pandas` 를 통해서 진행

In [6]:
import pandas as pd

## II. pandas로 1차원 데이터 다루기 - Series

### Series?

- 1-D labeled **array**
- 인덱스를 지정해줄 수 있음

In [14]:
s = pd.Series([1, 4, 9, 16, 25])

s

0     1
1     4
2     9
3    16
4    25
dtype: int64

In [15]:
t = pd.Series({'one' : 1, 'two' : 2, 'three':3, 'four': 4, 'five': 5})

t

one      1
two      2
three    3
four     4
five     5
dtype: int64

### Series + Numpy

- Series는 ndarray와 유사하다.

In [16]:
s[1]

4

In [17]:
t[1]

2

In [18]:
t[1:3]

two      2
three    3
dtype: int64

In [22]:
'''
이전의 파이썬의 List 때에는 이런 기능을 사용할 수 없었다. (함수를 만들어야됨)
pandas의 Series는 이런 기능을 지원하기 때문에 편리하게 데이터를 필터링해서 가져올 수 있다.
'''

# 조건을 통해 데이터를 가져올 수 있다.
s[s > s.median()]  # 자기 자신의 median(중앙값) 보다 큰 값들만 가지고 와라

3    16
4    25
dtype: int64

In [23]:
# 인덱스를 리스트화 해서 여러가지의 인덱스를 동시에 가져올 수 있다.
s[[3, 1, 4]]

3    16
1     4
4    25
dtype: int64

In [26]:
# Numpy의 함수 사용
import numpy as np

np.exp(s)  # 지수함수 사용 2^s[n] (n = 1,2,..,, n)

0    2.718282e+00
1    5.459815e+01
2    8.103084e+03
3    8.886111e+06
4    7.200490e+10
dtype: float64

In [27]:
# Series에 담긴 데이터 타입 확인
s.dtype

dtype('int64')

In [ ]:
'''
Series와 numpy와 유사한 부분이 많다.
'''

### Series + dict

- Series는 **dict**와 유사하다.

In [28]:
t

one      1
two      2
three    3
four     4
five     5
dtype: int64

In [29]:
t['one']

1

In [30]:
# Series에 값 추가
t['six'] = 6

t

one      1
two      2
three    3
four     4
five     5
six      6
dtype: int64

In [31]:
# 특정 키워드가 있는지 확인
'six' in t

True

In [32]:
'seven' in t

False

In [34]:
#t['seven']  # KeyError 발생

In [38]:
# 어떤 값을 가져오는데, 키가 없다면 아무 반응하지 않음 
# (옵션을 통해 키가 없을시 반환하는 값을 설정 할 수 있다.)
t.get('seven', 0)

0

In [39]:
t.get('six')

6

### Series에 이름 붙이기

- 'name' 속성을 가지고 있다.
- 처음 Series를 만들 때 이름을 붙일 수 있다.

In [44]:
s = pd.Series(np.random.randn(5), name='random_nums')

# 출력에 name의 항목이 생긴것을 볼 수 있다.
s

0    1.719511
1   -0.278304
2   -0.458638
3    2.200505
4    1.661865
Name: random_nums, dtype: float64

In [47]:
# name 접근 가능
s.name = '임의의 난수'

s

0    1.719511
1   -0.278304
2   -0.458638
3    2.200505
4    1.661865
Name: 임의의 난수, dtype: float64

## III. Pandas로 2차원 데이터 다루기 - dataframe

### dataframe??

- 2-D labeled **table** (쉽게 말해 엑셀의 표 생각하면 됨)
- 인덱스를 지정할 수 있다.

In [50]:
'''
dataframe은 리스트만 가지고는 2차원 데이터를 만들기 어렵다
따라서 딕셔너리로 만드는게 더 쉽다.
'''

d = {'height':[1,2,3,4], 'weight':[30, 40, 50, 60]}

df = pd.DataFrame(d)  # 주의 D, F가 대문자. pd.DataFrame은 데이터를 dataframe으로 만드는 함수

df

,height,weight
0,1,30
1,2,40
2,3,50
3,4,60


In [51]:
'''
dataframe은 여러가지 데이터 타입을 담을 수 있기 때문에 이를 확인해주는 작업이 중요
나중에 EDA에서 사용된다.

dtype 확인
numpy의 경우 : numpy.array.dtype
pandas인 경우 : 여러 컬럼이 존재하기 때문에 dtypes를 사용
'''

df.dtypes

height    int64
weight    int64
dtype: object

### From CSV to Dataframe

- comma Separated Value(CSV)를 DataFrame으로 생성해줄 수 있다.
- `.read_csv()` 를 이용

기존의 데이터를 dataframe화 시켜서 많이 사용한다.
대중적으로 csv 파일을 사용

In [53]:
# 동일 경로에 country_wise_latest.csv가 존재하면 : 
# read_csv() 매개변수로는 CSV가 있는 경로를 넣어준다.

covid = pd.read_csv("./country_wise_latest.csv")

covid

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,West Bank and Gaza,10621,78,3752,6791,152,2,0,0.73,35.33,2.08,8916,1705,19.12,Eastern Mediterranean
183,Western Sahara,10,1,8,1,0,0,0,10.00,80.00,12.50,10,0,0.00,Africa
184,Yemen,1691,483,833,375,10,4,36,28.56,49.26,57.98,1619,72,4.45,Eastern Mediterranean
185,Zambia,4552,140,2815,1597,71,1,465,3.08,61.84,4.97,3326,1226,36.86,Africa


### Pandas 활용 1. 일부분만 관찰하기

- 데이터가 너무 방대해서 다 참조하는 것이 힘들기 때문에

1. `head(n)` : 처음 n개의 데이터 참조

In [54]:
# 위에서부터 5개를 관찰하는 방법(함수)

covid.head(5)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


2. `tail(n)` : 마지막 n개의 데이터를 참조 

In [55]:
# 아래에서부터 5개를 관찰하는 방법(함수)

covid.tail(5)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
182,West Bank and Gaza,10621,78,3752,6791,152,2,0,0.73,35.33,2.08,8916,1705,19.12,Eastern Mediterranean
183,Western Sahara,10,1,8,1,0,0,0,10.00,80.00,12.50,10,0,0.00,Africa
184,Yemen,1691,483,833,375,10,4,36,28.56,49.26,57.98,1619,72,4.45,Eastern Mediterranean
185,Zambia,4552,140,2815,1597,71,1,465,3.08,61.84,4.97,3326,1226,36.86,Africa
186,Zimbabwe,2704,36,542,2126,192,2,24,1.33,20.04,6.64,1713,991,57.85,Africa


### Pandas 활용 2. 데이터 접근하기

- `df['column_name']` or `df.column_name`

In [59]:
# covid['WHO Region'] -> 제대로 접근 가능
covid['Active']

0      9796
1      1991
2      7973
3        52
4       667
       ... 
182    6791
183       1
184     375
185    1597
186    2126
Name: Active, Length: 187, dtype: int64

In [60]:
# column_name에 띄어쓰기가 있을 경우 제대로 접근할 수 없다.
# covid.WHO Region -> invalid syntax

covid.Active

0      9796
1      1991
2      7973
3        52
4       667
       ... 
182    6791
183       1
184     375
185    1597
186    2126
Name: Active, Length: 187, dtype: int64

### Honey Tip! DataFrame의 각 column은 "Series"다.

- Series에서 배웠던 것들을 활용가능

In [63]:
type(covid['Confirmed'])

pandas.core.series.Series

In [65]:
covid['Confirmed'][0]

36263

In [66]:
covid['Confirmed'][1:5]

1     4880
2    27973
3      907
4      950
Name: Confirmed, dtype: int64

### Pandas 활용 3. "조건"을 이용해서 데이터 접근하기

In [69]:
# 신규 확진자가 100명이 넘는 나라를 찾아보자
# "조건" 기능을 사용할 수 없으면 for문으로 하나씩 찾아봐야한다.

covid['New cases'] > 100  # 이 구문을 조건으로 사용

0       True
1       True
2       True
3      False
4      False
       ...  
182     True
183    False
184    False
185    False
186     True
Name: New cases, Length: 187, dtype: bool

In [71]:
covid[covid['New cases']>100].head(5)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
6,Argentina,167416,3059,72575,91782,4890,120,2057,1.83,43.35,4.21,130774,36642,28.02,Americas
8,Australia,15303,167,9311,5825,368,6,137,1.09,60.84,1.79,12428,2875,23.13,Western Pacific


In [72]:
# WHO 지역(WHO Region) 이 동남아시아인 나라 찾기
# WHO Region은 범주형(categori) 데이터이므로 이 범주 종류를 확인해줘야한다.

covid['WHO Region'].unique()

array(['Eastern Mediterranean', 'Europe', 'Africa', 'Americas',
       'Western Pacific', 'South-East Asia'], dtype=object)

In [74]:
# South-East Asia = 동남아시아인 걸로 확인

covid[covid['WHO Region'] == 'South-East Asia']

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
13,Bangladesh,226225,2965,125683,97577,2772,37,1801,1.31,55.56,2.36,207453,18772,9.05,South-East Asia
19,Bhutan,99,0,86,13,4,0,1,0.00,86.87,0.00,90,9,10.00,South-East Asia
27,Burma,350,6,292,52,0,0,2,1.71,83.43,2.05,341,9,2.64,South-East Asia
79,India,1480073,33408,951166,495499,44457,637,33598,2.26,64.26,3.51,1155338,324735,28.11,South-East Asia
80,Indonesia,100303,4838,58173,37292,1525,57,1518,4.82,58.00,8.32,88214,12089,13.70,South-East Asia
106,Maldives,3369,15,2547,807,67,0,19,0.45,75.60,0.59,2999,370,12.34,South-East Asia
119,Nepal,18752,48,13754,4950,139,3,626,0.26,73.35,0.35,17844,908,5.09,South-East Asia
158,Sri Lanka,2805,11,2121,673,23,0,15,0.39,75.61,0.52,2730,75,2.75,South-East Asia
167,Thailand,3297,58,3111,128,6,0,2,1.76,94.36,1.86,3250,47,1.45,South-East Asia
168,Timor-Leste,24,0,0,24,0,0,0,0.00,0.00,0.00,24,0,0.00,South-East Asia


### Pandas 활용 4. 행을 기준으로 데이터 접근하기


In [78]:
# 예시 데이터 - 도서관 정보

books_dict = {"Available" : [True, True, False], "Location":[102, 215, 323], "Genre":["Programming", "Physics", "Math"]}

books_df = pd.DataFrame(books_dict, index=['버그란 무엇인가', '두근두근 물리학', '미분해줘 홈즈'])

books_df

,Available,Location,Genre
버그란 무엇인가,True,102,Programming
두근두근 물리학,True,215,Physics
미분해줘 홈즈,False,323,Math


### 인덱스를 이용해서 가져오기 : `.loc[row, col]`

In [79]:
books_df.loc["버그란 무엇인가"]

Available           True
Location             102
Genre        Programming
Name: 버그란 무엇인가, dtype: object

In [83]:
# '미분해줘 홈즈' 책이 대출가능한지 확인하는 방법

books_df.loc["미분해줘 홈즈", "Available"]

False

### 숫자 인덱스를 이용해서 가져오기 : `.iloc[row_idx, col_idx]`

In [86]:
# 인덱스 0행의 인덱스 1열 가져오기

books_df.iloc[0, 1]

102

In [88]:
# 인덱스 1행의 인덱스 0~1열 가져오기

books_df.iloc[1, 0:2]

Available    True
Location      215
Name: 두근두근 물리학, dtype: object

## Pandas 활용 5. groupby


데이터 테이블을 3가지 과정을 거쳐서 더 좋은 결과를 얻는 것
1. Split : 특정한 "기준"을 바탕으로 DataFrame을 분할
2. Apply : 통계함수 - sum(), mean(), median(), ... 을 적용해서 각 데이터를 압축
3. Combine : Apply된 결과를 바탕으로 새로운 Series를 생성 (group_key : applied_value)

`groupby()`

In [89]:
covid.head(5)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [91]:
# WHO Region 별 확진자 수 

'''
1. covid에서 확진자 수 column만 추출한다.
2. 이를 covid의 WHO Region을 기준으로 groupby한다.
'''

covid_by_region = covid["Confirmed"].groupby(by=covid["WHO Region"])  # by 매개변수로 기준을 정해줌 (Series형태로 적어주는 것이 좋음)

covid_by_region  # 현재 Split만 적용된 형태

In [96]:
covid_by_region.sum()  # 통계함수 적용

# Americas와 Europe에 국가가 많아서 감염자 수가 많이 나온거 아닐까? 즉, 표본의 수가 많아서 높은 숫자가 나온거 아닐까?

WHO Region
Africa                    723207
Americas                 8839286
Eastern Mediterranean    1490744
Europe                   3299523
South-East Asia          1835297
Western Pacific           292428
Name: Confirmed, dtype: int64

In [95]:
# 국가당 감염자 수

covid_by_region.mean()  # sum() / 국가 수

# Europe의 경우 국가 당 숫자는 적다. 즉 위의 가설이 맞음
# South-East Asia가 숫자가 크게 나옴

WHO Region
Africa                    15066.812500
Americas                 252551.028571
Eastern Mediterranean     67761.090909
Europe                    58920.053571
South-East Asia          183529.700000
Western Pacific           18276.750000
Name: Confirmed, dtype: float64

## Mission:
### 1. covid 데이터에서 100 case 대비 사망률(`Deaths / 100 Cases`)이 가장 높은 국가는?

In [102]:
covid.head(5)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [111]:
covid["Deaths / 100 Cases"].groupby(by=covid["Country/Region"]).sum().idxmax()

'Yemen'

### 2. covid 데이터에서 신규 확진자가 없는 나라 중 WHO Region이 'Europe'를 모두 출력하면?  
Hint : 한 줄에 동시에 두가지 조건을 Apply하는 경우 Warning이 발생할 수 있습니다.

In [115]:
covid_by_None = covid[covid["New cases"]==0]
covid_by_None.head(5)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
14,Barbados,110,7,94,9,0,0,0,6.36,85.45,7.45,106,4,3.77,Americas
17,Belize,48,2,26,20,0,0,0,4.17,54.17,7.69,40,8,20.00,Americas
18,Benin,1770,35,1036,699,0,0,0,1.98,58.53,3.38,1602,168,10.49,Africa
24,Brunei,141,3,138,0,0,0,0,2.13,97.87,2.17,141,0,0.00,Western Pacific
27,Burma,350,6,292,52,0,0,2,1.71,83.43,2.05,341,9,2.64,South-East Asia


In [118]:
covid_by_None[covid_by_None["WHO Region"] == "Europe"]["Country/Region"].unique()

array(['Estonia', 'Holy See', 'Latvia', 'Liechtenstein', 'Monaco',
       'San Marino', 'Spain'], dtype=object)

### 3. 다음 [데이터](https://www.kaggle.com/neuromusic/avocado-prices)를 이용해 각 Region별로 아보카도가 가장 비싼 평균가격(AveragePrice)을 출력하면?

In [119]:
import pandas as pd

avocado = pd.read_csv("./avocado.csv")
avocado.head(5)

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [124]:
avocado["AveragePrice"].groupby(by=avocado["region"]).max()

region
Albany                 2.13
Atlanta                2.75
BaltimoreWashington    2.28
Boise                  2.79
Boston                 2.19
BuffaloRochester       2.57
California             2.58
Charlotte              2.83
Chicago                2.30
CincinnatiDayton       2.20
Columbus               2.22
DallasFtWorth          1.90
Denver                 2.16
Detroit                2.08
GrandRapids            2.73
GreatLakes             1.98
HarrisburgScranton     2.27
HartfordSpringfield    2.68
Houston                1.92
Indianapolis           2.10
Jacksonville           2.99
LasVegas               3.03
LosAngeles             2.44
Louisville             2.29
MiamiFtLauderdale      3.05
Midsouth               2.17
Nashville              2.24
NewOrleansMobile       2.32
NewYork                2.65
Northeast              2.31
NorthernNewEngland     1.96
Orlando                2.87
Philadelphia           2.45
PhoenixTucson          2.62
Pittsburgh             1.83
Plains       